In [81]:
import warnings
warnings.filterwarnings("ignore")

In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from keras.callbacks import ProgbarLogger
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier #!pip install scikeras
from keras.callbacks import ProgbarLogger

In [119]:
df = pd.read_csv('drug200.csv')
set(df[df.columns[-1]]) #df['Drug']

{'DrugY', 'drugA', 'drugB', 'drugC', 'drugX'}

In [120]:
#df['Embarked'] = df['Embarked'].fillna(method='ffill')

# One-hot encode 'Embarked' column
onehot_encoder = OneHotEncoder(sparse=False, drop='if_binary')
onehot_encoded = onehot_encoder.fit_transform(df[['Drug']])
df[
    ['DrugY', 'drugA', 'drugB', 'drugC', 'drugX']
    ] = onehot_encoded

# Create a mapping dictionary for regrouping
embarked_mapping = {'DrugY': 1, 'drugA': 2, 'drugB': 3, 'drugC': 4, 'drugX': 5}

# Create 'Embarked' column by mapping the encoded values
df['Drug'] = df[['DrugY', 'drugA', 'drugB', 'drugC', 'drugX']].idxmax(axis=1)
df['Drug'] = df['Drug'].map(embarked_mapping)

# Drop the one-hot encoded columns
df.drop(['DrugY', 'drugA', 'drugB', 'drugC', 'drugX'], axis=1, inplace=True)

In [121]:
set(df[df.columns[-1]])

{1, 2, 3, 4, 5}

In [122]:
set(df["Cholesterol"])

{'HIGH', 'NORMAL'}

In [123]:
sex_mapping = {'HIGH': 0, 'LOW': 1,'NORMAL':2}
df['BP'] = df['BP'].map(sex_mapping)

In [124]:
sex_mapping = {'F': 0, 'M': 1}
df['Sex'] = df['Sex'].map(sex_mapping)

In [125]:
sex_mapping = {'HIGH': 0, 'NORMAL': 1}
df['Cholesterol'] = df['Cholesterol'].map(sex_mapping)

In [126]:
df.head(5)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,1
1,47,1,1,0,13.093,4
2,47,1,1,0,10.114,4
3,28,0,2,0,7.798,5
4,61,0,1,0,18.043,1


In [127]:
# Separate the features (X) and target variable (y)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [128]:
preprocessor = ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'), slice(0, None))])

# Preprocess the numerical columns
X = preprocessor.fit_transform(X)

In [129]:
(len(df[df["Drug"] == 1]),len(df[df["Drug"] == 2]),len(df[df["Drug"] == 3]),len(df[df["Drug"] == 4]),len(df[df["Drug"] == 5]))

(91, 23, 16, 16, 54)

In [130]:
over = RandomOverSampler()
X,y = over.fit_resample(X,y)
data = np.concatenate((X, y.reshape(-1, 1)), axis=1)


data = np.hstack(
    (X,np.reshape(y,(-1,1)))
)
df = pd.DataFrame(data,columns=df.columns)


In [141]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [142]:
(len(df[df["Drug"] == 1]),len(df[df["Drug"] == 2]),len(df[df["Drug"] == 3]),len(df[df["Drug"] == 4]),len(df[df["Drug"] == 5]))

(91, 91, 91, 91, 91)

In [155]:
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=.1,random_state=0)
X_valid,X_test,y_valid,y_test = train_test_split(X_temp,y_temp,test_size=.2,random_state=0)

In [156]:
# Check for missing or NaN values in X_train (numpy array)
missing_values = np.isnan(X_train).sum() if isinstance(X_train, np.ndarray) else 0
print(missing_values)

0


In [159]:
# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Reshape the target data to have a single column
y_train_reshaped = y_train.reshape(-1, 1)
y_valid_reshaped = y_valid.reshape(-1, 1)
y_test_reshaped = y_test.reshape(-1, 1)


# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_reshaped)
y_valid_encoded = encoder.transform(y_valid_reshaped)
y_test_encoded = encoder.transform(y_test_reshaped)

In [163]:
loss_ = [tf.keras.losses.CategoricalCrossentropy(),
         tf.keras.losses.MSE]

optimizers_ = [tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
               tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001)]



In [166]:
for i in loss_:
    for j in optimizers_:
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(16, activation="relu"))
        model.add(tf.keras.layers.Dropout(0.4))
        model.add(tf.keras.layers.Dense(16, activation="relu"))
        model.add(tf.keras.layers.Dropout(0.4))
        model.add(tf.keras.layers.Dense(16, activation="relu"))
        model.add(tf.keras.layers.Dropout(0.4))
        model.add(tf.keras.layers.Dense(5, activation="softmax"))

        model.compile(
            optimizer=j,
            loss=i,
            metrics=['accuracy']
        )

        model.fit(X_train, y_train_encoded, batch_size=64, epochs=40, validation_data=(X_valid, y_valid_encoded), verbose=0, callbacks=[ProgbarLogger(count_mode='steps')])

        accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)[1]
        print(f"accuracy = {accuracy}")


accuracy = 0.6000000238418579
accuracy = 0.6000000238418579
accuracy = 0.699999988079071
accuracy = 0.20000000298023224
